# WF

In [34]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *
import xmltodict
import colabexts.utils as colabexts_utils


import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

import logging
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig( level=logging.INFO,
        format='%(asctime)s %(name)s %(levelname)s: %(message)s',
        #handlers=[ logging.FileHandler("/tmp/stream.log"), logging.StreamHandler()],
        handlers=[ logging.StreamHandler()],
)
logger = logging.getLogger( "geoapp" )
logger.info( f"Testing logging" )

2024-07-12 15:44:23,672 geoapp INFO: Testing logging


# WF_services

In [49]:
%%writefile ../wf_services.py
#!/usr/bin/env python 

#*** DO NOT EDIT - GENERATED FROM tseries/notebooks/tseries_services.ipynb ****

import sys, os,  glob, logging, datetime, json
from  mangorest.mango import webapi
from colabexts import utils as colabexts_utils
import pandas as pd
import xmltodict
import colabexts.utils as colabexts_utils

sys.path.append("../..")

MBASE  = "/opt/data/data/workflows/"
logger = logging.getLogger( "geoapp" )

#---------------------------------------------------------------------------------    
@webapi("/wf/save_wf")
def save_wf( name, jsn, xml, **kwargs):
    name = name or "default"
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]
        
    if not os.path.exists(MBASE ):
        os.makedirs(MBASE)
    
    with open(f"{MBASE}/{name}.xml",  "w") as f:   f.write(xml.strip())
    if ( len(jsn) < 30 ):
        jsn = xmltodict.parse(xml,attr_prefix='', cdata_key='#text')
        jsn = json.dumps(jsn, indent=4)

    with open(f"{MBASE}/{name}.json", "w") as f:   f.write(jsn.strip())

    return f"Saved "    
#---------------------------------------------------------------------------------    
@webapi("/wf/get_wf")
def get_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    xml = open(f"{MBASE}/{name}.xml",  "r").read()
    ret = dict(json="", xml=xml)

    return ret  
#------------------------------------------------------------------------------
@webapi("/wf/delete_wf")
def delete_wf(name="", **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    logger.info(f"DELETE {name}")
    
    dst = f"{MBASE}/{name}.xml"
    if ( os.path.exists(dst) ):  os.remove(dst)

    dst = f"{MBASE}/{name}.json"
    if ( os.path.exists(dst) ):  os.remove(dst)

    return f"Deleted {dst}."
#------------------------------------------------------------------------------
@webapi("/wf/getall_wfs")
def getAllWorkflows( nrows=10000, patt='*.xml', **kwargs):
    
    files = glob.glob(f"{MBASE}{patt}")
    files.sort(key=os.path.getmtime, reverse=True)

    ret = {
        "name": "workflows",
        "columns": ["name"],
        "values": [[f[len(MBASE):]] for f in files][0:nrows]
    }
    return ret
#---------------------------------------------------------------------------------    
@webapi("/wf/run_wf")
def run_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    wfdef  = f"/opt/data/data/workflows/{name}.xml"
    buf=open(wfdef,"r").read()
    wf = xmltodict.parse(buf,attr_prefix='', cdata_key='#text')


    wfsh="#Execute\n"
    for c in wf['mxGraphModel']['root']['mxCell']:
        w = colabexts_utils.Map(c)
        s = w.text or ""
        wfsh += f"#Processing ID: {w.id}, Value: {w.value} => Cmd: {s}\n"
        if (not s):
            continue
        wfsh += f"{s}\n"

    open("/tmp/run.sh","w").write(wfsh)

    return f"Running /tmp/run.sh {wfsh} "

In [50]:
run_wf("sada")

'Running /tmp/run.sh #Execute\n#Processing ID: 0, Value: None => Cmd: \n#Processing ID: 1, Value: None => Cmd: \n#Processing ID: 2, Value: Start => Cmd: \n#Processing ID: 3, Value: Process Data! => Cmd: \n#Processing ID: 4, Value: Build Model! => Cmd: \n#Processing ID: 5, Value: Send Email! OK => Cmd: SADANNADA\nSADANNADA\n#Processing ID: 6, Value:  => Cmd: \n#Processing ID: 7, Value:  => Cmd: \n#Processing ID: 8, Value:  => Cmd: \n#Processing ID: 9, Value: None => Cmd: \n#Processing ID: 10, Value: None => Cmd: \n#Processing ID: 11, Value: System Admin => Cmd: Validate and Verify\xa0Is Approved?SADA\nValidate and Verify\xa0Is Approved?SADA\n#Processing ID: 12, Value:  => Cmd: \n#Processing ID: 13, Value: S => Cmd: START\nSTART\n#Processing ID: 14, Value: None => Cmd: \n '